In [19]:
# import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import re
import json
import requests
import numpy as np

In [4]:
# Start BLS CSV Extract/Transform
#Create dataframe for Winchester
winchester_df = pd.read_excel('SeriesReport-20190415194424_44baf2.xlsx', skiprows=10, index_col=0, na_values=['NA'])
winchester_df=winchester_df.rename(index=str, columns={"labor force": "labor_force", "unemployment rate": "unemployment_rate"})
winchester_df["city"] = "Winchester"



#Create dataframe for Staunton
staunton_df = pd.read_excel('SeriesReport-20190415194440_c6f6e8.xlsx', skiprows=10, index_col=0, na_values=['NA'])
staunton_df=staunton_df.rename(index=str, columns={"labor force": "labor_force", "unemployment rate": "unemployment_rate"})
staunton_df["city"] = "Staunton"



#Create dataframe for Blacksburg
blacksburg_df = pd.read_excel('SeriesReport-20190415194501_2b534f.xlsx', skiprows=10, index_col=0, na_values=['NA'])
blacksburg_df=blacksburg_df.rename(index=str, columns={"labor force": "labor_force", "unemployment rate": "unemployment_rate"})
blacksburg_df["city"] = "Blacksburg"


#Create dataframe for Virginia Beach
beach_df = pd.read_excel('SeriesReport-20190415194437_539c01.xlsx', skiprows=10, index_col=0, na_values=['NA'])
beach_df=beach_df.rename(index=str, columns={"labor force": "labor_force", "unemployment rate": "unemployment_rate"})
beach_df["city"] = "Virginia_Beach"



#Create dataframe for Charlottesville
charlottesville_df = pd.read_excel('SeriesReport-20190415194458_0390c9.xlsx', skiprows=10, index_col=0, na_values=['NA'])
charlottesville_df=charlottesville_df.rename(index=str, columns={"labor force": "labor_force", "unemployment rate": "unemployment_rate"})
charlottesville_df["city"] = "Charlottesville"


#Create dataframe for Lynchburg
lynchburg_df = pd.read_excel('SeriesReport-20190415194451_9bfb85.xlsx', skiprows=10, index_col=0, na_values=['NA'])
lynchburg_df=lynchburg_df.rename(index=str, columns={"labor force": "labor_force", "unemployment rate": "unemployment_rate"})
lynchburg_df["city"] = "Lynchburg"


#Create dataframe for Roanoke
roanoke_df = pd.read_excel('SeriesReport-20190415194451_9bfb85.xlsx', skiprows=10, index_col=0, na_values=['NA'])
roanoke_df=roanoke_df.rename(index=str, columns={"labor force": "labor_force", "unemployment rate": "unemployment_rate"})
roanoke_df["city"] = "Roanoke"


#Create dataframe for Richmond
richmond_df = pd.read_excel('SeriesReport-20190415194448_7b931d.xlsx', skiprows=10, index_col=0, na_values=['NA'])
richmond_df=richmond_df.rename(index=str, columns={"labor force": "labor_force", "unemployment rate": "unemployment_rate"})
richmond_df["city"] = "Richmond"


#Create dataframe for Harrisonburg
harrisonburg_df = pd.read_excel('SeriesReport-20190415194454_4a720b.xlsx', skiprows=10, index_col=0, na_values=['NA'])
harrisonburg_df=harrisonburg_df.rename(index=str, columns={"labor force": "labor_force", "unemployment rate": "unemployment_rate"})
harrisonburg_df["city"] = "Harrisonburg"


#Combined all dataframes into one
combined_df = pd.concat([richmond_df,roanoke_df,blacksburg_df,beach_df,charlottesville_df,\
                   lynchburg_df,roanoke_df,richmond_df,harrisonburg_df])
combined_df.head()

,Period,labor_force,employment,unemployment,unemployment_rate,city
Year,,,,,,
2009,Jan,625783,585818,39965,6.4,Richmond
2009,Feb,625547,581269,44278,7.1,Richmond
2009,Mar,626819,581541,45278,7.2,Richmond
2009,Apr,627293,584134,43159,6.9,Richmond
2009,May,628070,580927,47143,7.5,Richmond


In [7]:
# Create database connection
rds_connection_string = "<username>:<password>@127.0.0.1/etl_project_db"
engine = create_engine(f'mysql://{rds_connection_string}', encoding='utf-8')

In [30]:
# Confirm tables
engine.table_names()

['bls_data', 'city_data', 'indeed_data']

In [10]:
# Load combined_df to the mysql database
combined_df.to_sql(name='bls_data', con=engine, if_exists='append', index=True)

In [11]:
# view combined_df in mysql database
pd.read_sql_query('select * from bls_data', con=engine).head()

,Year,Period,Labor_force,Employment,Unemployment,Unemployment_rate,Id,City
0,2009,Jan,625783,585818,39965,6.0,1,Richmond
1,2009,Feb,625547,581269,44278,7.0,2,Richmond
2,2009,Mar,626819,581541,45278,7.0,3,Richmond
3,2009,Apr,627293,584134,43159,7.0,4,Richmond
4,2009,May,628070,580927,47143,8.0,5,Richmond


In [15]:
# Start Indeed webscrape
# set up cities to search
cities = ['Winchester', 'Staunton', 'Blacksburg', 'Virginia+Beach',
          'Charlottesville', 'Lynchburg', 'Roanoke', 'Richmond', 'Harrisonburg']

# create two lists for a list of lists for easy panda conversion
joblists = []
job = []
sep = ','  # comma separator defined for cleaning location variable below

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F9FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def clean_string(stuff):
    return ' '.join(
        re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(RT @[\w_]+: )",
               " ", stuff).split())

for city in cities:

    # starting url for each city
    url = "https://www.indeed.com/jobs?q=&l=" + city + "%2C+VA"
    response = requests.get(url)

    # record counter for website started outside at zero for immediate increment and reset
    counter = 0

    # loop through 500 records (50 x 10)
    # if there are not enough records, the script will continue without error
    for x in range(0, 50):

        # implement bs
        soup = BeautifulSoup(response.text, 'html.parser')

        # search bs
        results = soup.find_all('div', class_="jobsearch-SerpJobCard")

        for result in results:
            try:
                # find job title
                title = result.find('a', class_="turnstileLink").text.strip()
                title = clean_string(remove_emoji(title))
                # find location of job
                location = result.find('div', class_="location").text.strip()
                # strip away just the city
                location = location.split(sep, 1)[0]
                # if there is a job title and a location then...
                if (title and location):
                    job = [title, location]  # record the job and location
                    joblists.append(job)  # add the job and location to the list
                    job = []  # clear the job to go to the next
                    #  print(title)
                    #  print(location)

            except AttributeError:
                continue  # continuing without recording job and location

        counter += 10  # increment the next set of records
        # find the next set of records for the city
        url = "https://www.indeed.com/jobs?q=&l=" + city + "%2C+VA&start=" + str(counter)
        response = requests.get(url)

# drop it into a dataframe
new_df = pd.DataFrame(joblists, columns=['job_title', 'city'])
# bit of head.
new_df.head()

,job_title,city
0,Audio Transcriptionist,Virginia
1,Grill Server or Course Beverage Cart Person,Winchester
2,mechanical electrical assembly,Winchester
3,Open Interviews Hourly Associates 150 Sign On ...,Winchester
4,Executive Secretary,Winchester


In [17]:
# load webscraped indeed data to mysql database
new_df.to_sql(name='indeed_data', con=engine, if_exists='append', index=False)

In [18]:
# view web-scraped indeed data in mysql database
pd.read_sql_query('select * from indeed_data', con=engine).head()

,job_title,city
0,Audio Transcriptionist,Virginia
1,Grill Server or Course Beverage Cart Person,Winchester
2,mechanical electrical assembly,Winchester
3,Open Interviews Hourly Associates 150 Sign On ...,Winchester
4,Executive Secretary,Winchester


In [20]:
# Start API Process
req = requests.request('GET', 'https://api.census.gov/data/timeseries/qwi/se?get=year,quarter,EmpEnd&for=county:*&in=state:51&time=2017-Q1&key=<api key>')
data2017q1 = req.json()
df2017q1 = pd.DataFrame.from_records(data2017q1[1:], columns=data2017q1[0])

#d = {'2017q1': df2017q1}
#d['2017q1'].head()

In [21]:
years = ["2017", "2016", "2015", "2014", "2013", "2012", "2011", "2010"]
quarters = ["Q1", "Q2", "Q3", "Q4"]  
dfDic = {}
for year in years: 
    for quarter in quarters:
        req = requests.request('GET', f"https://api.census.gov/data/timeseries/qwi/se?get=year,quarter,EmpEnd&for=county:*&in=state:51&time={year}-{quarter}&key=<api key>"
        datadf = pd.DataFrame.from_records(datalist[1:], columns=datalist[0])
        dfDic[f'{year}' + f'{quarter}'] = datadf

In [22]:
dfDic['2016Q3'].head()

,year,quarter,EmpEnd,time,state,county
0,2016,3,11675,2016-Q3,51,001
1,2016,3,49038,2016-Q3,51,003
2,2016,3,3846,2016-Q3,51,005
3,2016,3,2352,2016-Q3,51,007
4,2016,3,8677,2016-Q3,51,009


In [23]:
final_df = pd.concat(dfDic, ignore_index=True)

In [29]:
city_data = final_df.loc[final_df['county'].isin(["840", "790", "101", "810", "540", "680", "760", "770", "660"])]

In [25]:
city_data["state"] = [x.strip().replace('51', 'Virginia') for x in city_data["state"]]
city_data["county"] = [x.strip().replace('840', 'Winchester') for x in city_data["county"]]
city_data["county"] = [x.strip().replace('790', 'Staunton') for x in city_data["county"]]
city_data["county"] = [x.strip().replace('101', 'Blacksburg') for x in city_data["county"]]
city_data["county"] = [x.strip().replace('810', 'Virginia Beach') for x in city_data["county"]]
city_data["county"] = [x.strip().replace('540', 'Charlottesville') for x in city_data["county"]]
city_data["county"] = [x.strip().replace('680', 'Lynchburg') for x in city_data["county"]]
city_data["county"] = [x.strip().replace('760', 'Richmond') for x in city_data['county']]
city_data["county"] = [x.strip().replace('770','Roanoke') for x in city_data['county']]
city_data["county"] = [x.strip().replace('660', 'Harrisonburg') for x in city_data['county']]

city_data.columns = [x.strip().replace('EmpEnd', 'Employment') for x in city_data.columns]

C:\Users\jcfis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\jcfis\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\jcfis\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [26]:
city_data.head(10)

,year,quarter,Employment,time,state,county
49,2010,1,3588,2010-Q1,Virginia,Blacksburg
98,2010,1,32963,2010-Q1,Virginia,Charlottesville
110,2010,1,33485,2010-Q1,Virginia,Harrisonburg
113,2010,1,45952,2010-Q1,Virginia,Lynchburg
124,2010,1,147195,2010-Q1,Virginia,Richmond
125,2010,1,66128,2010-Q1,Virginia,Roanoke
127,2010,1,11642,2010-Q1,Virginia,Staunton
129,2010,1,159366,2010-Q1,Virginia,Virginia Beach
132,2010,1,24773,2010-Q1,Virginia,Winchester
182,2010,2,3602,2010-Q2,Virginia,Blacksburg


In [27]:
# load API data to mysql database
city_data.to_sql(name='city_data', con=engine, if_exists='append', index=False)

In [28]:
# view web-scraped indeed data in mysql database
pd.read_sql_query('select * from city_data', con=engine).head()

,year,quarter,Employment,time,state,county
0,2010,1,3588,2010-Q1,Virginia,Blacksburg
1,2010,1,32963,2010-Q1,Virginia,Charlottesville
2,2010,1,33485,2010-Q1,Virginia,Harrisonburg
3,2010,1,45952,2010-Q1,Virginia,Lynchburg
4,2010,1,147195,2010-Q1,Virginia,Richmond
